In [1]:
import glob
import os
import pandas as pd
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from neo4j import GraphDatabase
import torch
from transformers import AutoModel, AutoTokenizer
import stanza
import numpy as np
from scipy.spatial.distance import cosine
import py_vncorenlp
import re, unicodedata
import jnius_config

# Cấu hình JVM trước khi import bất kỳ module nào kích hoạt JVM
jnius_config.add_options('-Xmx2g')

# Khởi tạo VnCoreNLP (đảm bảo file VnCoreNLP-1.2.jar nằm trong thư mục hiện tại)
model_vn = py_vncorenlp.VnCoreNLP(save_dir='.')

# Load PhoBERT để lấy embedding
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Load Stanza pipeline (nếu cần)
stanza.download("vi")
nlp = stanza.Pipeline("vi")

print("Mô hình NLP đã được tải xong.")


/Users/anne/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-05 17:02:40 INFO  WordSegmenter:24 - Loading Word Segmentation model
2025-04-05 17:02:41 INFO  PosTagger:23 - Loading POS Tagging model
2025-04-05 17:02:43 INFO  NerRecognizer:34 - Loading NER model
2025-04-05 17:02:57 INFO  DependencyParser:32 - Loading Dependency Parsing model


/Users/anne/.pyenv/versions/3.10.12/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-04-05 17:03:04 INFO: Downloaded file to /Users/anne/stanza_resources/resources.json
2025-04-05 17:03:04 INFO: Downloading default packages for language: vi (Vietnamese) ...
2025-04-05 17:03:05 INFO: File exists: /Users/anne/stanza_resources/vi/default.zip
2025-04-05 17:03:08 INFO: Finished downloading models and saved to /Users/anne/stanza_resources
2025-04-05 17:03:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-04-05 17:03:08 INFO: Downloaded file to /Users/anne/stanza_resources/resources.json
2025-04-05 17:03:10 INFO: Loadin

Mô hình NLP đã được tải xong.


In [5]:
def get_embedding(text):
    """
    Lấy embedding từ PhoBERT cho một câu hoặc từ.
    """
    # Tokenize & xử lý max_length
    input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=256)
    input_ids = torch.tensor([input_ids])  # Thêm batch dimension
    
    with torch.no_grad():
        outputs = phobert(input_ids)
    
    # Trả về vector trung bình của tất cả token
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()

def clean_token(token):
    """
    Chuẩn hóa token: 
    - Thay thế "_" bằng khoảng trắng
    - Xóa ký tự đặc biệt không cần thiết
    - Giữ nguyên chữ hoa với tên riêng (PER)
    """
    token = token.replace("_", " ")  # VnCoreNLP thường dùng _ để nối từ ghép
    token = re.sub(r'[^\w\s]', '', token)  # Xóa ký tự đặc biệt
    return token.strip()

def extract_entities_vn(text):
    """
    Trích xuất thực thể có tên từ văn bản bằng VnCoreNLP.
    """
    annotations = model_vn.annotate_text(text)
    entities = set()
    
    for sent in annotations.values():
        for token in sent:
            ner_label = token.get("nerLabel")
            word = clean_token(token.get("wordForm"))

            if ner_label != "O":  # Nếu là thực thể có tên
                entities.add(word)
    
    return list(entities)

def extract_pos_tags_vn(text):
    """
    Trích xuất POS từ câu, giữ nguyên từ gốc để tra cứu chính xác.
    """
    annotations = model_vn.annotate_text(text)
    pos_dict = {}

    for sent in annotations.values():
        for token in sent:
            word = clean_token(token.get("wordForm"))
            pos_tag = token.get("posTag")

            if word not in pos_dict:  # Tránh trùng lặp
                pos_dict[word] = pos_tag
    
    return pos_dict
def clear_kg(tx):
    tx.run("MATCH (n) DETACH DELETE n")

def add_entity_to_kg(tx, entity, embedding, pos_tag):
    entity = entity.lower()  # Chuyển thành chữ thường
    query = """
    MERGE (n:Entity {name: $entity})
    SET n.embedding = $embedding, n.pos_tag = $pos_tag
    """
    tx.run(query, entity=entity, embedding=embedding.tolist(), pos_tag=pos_tag)
print("Định nghĩa các hàm xử lý hoàn tất.")


Định nghĩa các hàm xử lý hoàn tất.


In [3]:
with open("trainCorpus.dat", "r", encoding="utf-8") as f:
    corpus_lines = [line.strip() for line in f if line.strip()]

print("Đã đọc {} dòng từ trainCorpus.dat".format(len(corpus_lines)))

Đã đọc 100124 dòng từ trainCorpus.dat


In [ ]:
corpus_entities = set()
unique_pos = defaultdict(set)  # Lưu từ -> nhiều POS

for line in corpus_lines:
    annotations = model_vn.annotate_text(line)  # GỌI DUY NHẤT 1 LẦN
    
    for sent in annotations.values():
        for token in sent:
            word = clean_token(token.get("wordForm"))
            ner_label = token.get("nerLabel")
            pos_tag = token.get("posTag")

            if ner_label != "O":  # Nếu là thực thể có tên
                corpus_entities.add(word)

            if word:  # Lưu POS cho từ
                unique_pos[word].add(pos_tag)

print("Tổng số thực thể trích xuất:", len(corpus_entities))
print("Tổng số từ có POS:", len(unique_pos))
print("Ví dụ 10 từ có POS:", [(k, list(v)) for k, v in list(unique_pos.items())[:10]])

🔍 Tổng số thực thể trích xuất: 14815
Tổng số từ có POS: 44136
Ví dụ 10 từ có POS: [('Các', ['Np', 'L']), ('tín đồ', ['N']), ('tôn giáo', ['N']), ('tích cực', ['A']), ('tham gia', ['V']), ('hoạt động', ['V', 'N']), ('xã hội', ['N']), ('Tín đồ', ['N']), ('các', ['L']), ('hăng hái', ['A'])]


In [ ]:
VALID_POS_TAGS = {"N", "V", "A", "P", "B"}  # Danh từ, động từ, tính từ, đại từ, trạng từ

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "123123123"
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

# Xóa dữ liệu cũ
with driver.session() as session:
    session.execute_write(clear_kg)

valid_entities = 0

with driver.session() as session:
    for entity in corpus_entities:
        embedding = get_embedding(entity)
        pos_tags = unique_pos.get(entity, set())  # Lấy POS tags, trả về set rỗng nếu không có
        
        # Nếu pos_tags là rỗng hoặc không hợp lệ, bỏ qua thực thể
        if not pos_tags or all(tag not in VALID_POS_TAGS for tag in pos_tags):
            continue  # Bỏ qua thực thể nếu không có POS tag hợp lệ
        
        # Lọc các POS tags hợp lệ
        filtered_pos_tags = {tag for tag in pos_tags if tag in VALID_POS_TAGS}
        
        # Chuyển các POS tags hợp lệ thành chuỗi
        pos_tag_str = ", ".join(filtered_pos_tags)

        # Chèn thực thể vào Neo4j
        session.execute_write(add_entity_to_kg, entity, embedding, pos_tag_str)
        valid_entities += 1

print(f"Đã chèn các thực thể vào Neo4j.")


✅ Đã chèn 2119 thực thể vào Neo4j.


In [13]:
# Danh sách các nhóm từ (POS tags) và định nghĩa của chúng
pos_groups = {
    "Danh từ": "Các từ chỉ người, vật, địa điểm, sự việc hoặc khái niệm, chẳng hạn như giáo sư, trường học, Hà Nội.",
    "Động từ": "Các từ chỉ hành động hoặc trạng thái, chẳng hạn như ăn, học, làm việc.",
    "Tính từ": "Các từ chỉ đặc điểm, tính chất của người, vật, hoặc sự việc, chẳng hạn như đẹp, nhanh, hăng hái.",
    "Phó từ": "Các từ chỉ mức độ hoặc cách thức của hành động, chẳng hạn như rất, hoàn toàn, mạnh mẽ.",
    "Liên từ": "Các từ dùng để nối các phần trong câu, chẳng hạn như và, nhưng, hoặc.",
    "Giới từ": "Các từ chỉ quan hệ không gian, thời gian, hoặc mối quan hệ giữa các đối tượng, chẳng hạn như tại, trong, với.",
    "Đại từ": "Các từ thay thế danh từ hoặc cụm danh từ, chẳng hạn như tôi, bạn, họ.",
    "Số từ": "Các từ chỉ số lượng, mức độ, hoặc thứ tự, chẳng hạn như một, hai, ba.",
    "Thán từ": "Các từ bộc lộ cảm xúc, như ôi, wow, ah.",
    "Từ định danh": "Các từ xác định hoặc làm rõ danh từ, chẳng hạn như cái, này, đó."
}

# Hàm để tạo các node cho nhóm từ
def create_pos_nodes(tx):
    for group, definition in pos_groups.items():
        query = f"""
        CREATE (n:POSGroup {{name: '{group}', definition: '{definition}'}})
        """
        tx.run(query)

# Tạo các node cho các nhóm từ
with driver.session() as session:
    session.write_transaction(create_pos_nodes)

print("Các node nhóm từ đã được tạo thành công!")

Các node nhóm từ đã được tạo thành công!


/var/folders/13/t8vndqp931b5g_jv43m0b3600000gn/T/ipykernel_84630/283123153.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_pos_nodes)


In [18]:
# Lấy danh sách các thực thể có trong Neo4j
with driver.session() as session:
    result = session.run("MATCH (e:Entity) RETURN e.name AS Entity")
    existing_entities = [record["Entity"] for record in result]
    print(f"Tổng số thực thể trong Neo4j: {len(existing_entities)}")

Tổng số thực thể trong Neo4j: 1632


In [20]:
# Hàm nối thực thể vào nhóm POS
def link_entities_to_groups(tx, entity, pos_tags):
    for pos_tag in pos_tags:
        # Xác định nhóm tương ứng với POS tag
        if pos_tag == "N":
            group_name = "Danh từ"
        elif pos_tag == "V":
            group_name = "Động từ"
        elif pos_tag == "A":
            group_name = "Tính từ"
        elif pos_tag == "P":
            group_name = "Phó từ"
        elif pos_tag == "B":
            group_name = "Liên từ"
        else:
            continue

        # Tạo mối quan hệ giữa thực thể và nhóm
        query = f"""
        MATCH (e:Entity {{name: '{entity}'}})
        MATCH (g:POSGroup {{name: '{group_name}'}})
        MERGE (e)-[:BELONGS_TO]->(g)
        """
        tx.run(query)

# Nối thực thể vào các nhóm POS
valid_entities = 0
with driver.session() as session:
    for entity in existing_entities:
        pos_tags = unique_pos.get(entity, {"UNK"})  # Lấy POS tag của thực thể

        # Chỉ nối nếu thực thể có POS tags hợp lệ
        filtered_pos_tags = {tag for tag in pos_tags if tag in VALID_POS_TAGS}

        # Nối thực thể với các nhóm dựa trên POS tags hợp lệ
        with session.begin_transaction() as tx:
            link_entities_to_groups(tx, entity, filtered_pos_tags)

            valid_entities += 1

print(f"✅ Đã nối {valid_entities} thực thể vào các nhóm thành công!")


✅ Đã nối 1632 thực thể vào các nhóm thành công!


In [26]:
# Hàm tạo các nhóm Hypernym
def create_hypernym_groups(tx, hypernyms):
    for hypernym in hypernyms:
        query = f"""
        MERGE (h:HypernymGroup {{name: '{hypernym}'}})
        """
        tx.run(query)

# Danh sách các hypernyms (có thể mở rộng thêm)
hypernyms = [
    "Động vật", "Cây cối", "Phương tiện giao thông", "Thiết bị điện tử", "Công ty", "Quốc gia"
]

# Chèn các nhóm Hypernym vào Neo4j
with driver.session() as session:
    with session.begin_transaction() as tx:
        create_hypernym_groups(tx, hypernyms)

print("✅ Đã tạo các nhóm hypernym trong Neo4j!")

✅ Đã tạo các nhóm hypernym trong Neo4j!


In [45]:
import requests
from bs4 import BeautifulSoup

def extract_description_from_vdict(word):
    # URL của trang VDict tương ứng với từ cần tra
    url = f"https://vdict.com/{word},3,0,0.html"  # Sử dụng từ điển Vietnamese-Vietnamese (dict=3)
    
    # Gửi yêu cầu GET tới trang web
    headers = {'User-Agent': 'Mozilla/5.0'}  # Thêm header để tránh bị chặn
    response = requests.get(url, headers=headers)
    
    # Kiểm tra xem yêu cầu có thành công không
    if response.status_code != 200:
        print(f"Lỗi khi tải trang {url}")
        return None

    # Phân tích trang HTML bằng BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Tìm phần định nghĩa (có thể là friendly hoặc academic)
    descriptions = []

    # Hàm phụ để giữ khoảng cách tự nhiên trong văn bản
    def clean_text(element):
        # Lấy văn bản và giữ khoảng cách tự nhiên
        text = ' '.join(element.stripped_strings)
        return text.strip()

    # Tìm phần friendlyDefinition (nếu có)
    friendly_div = soup.find('div', id='friendlyDefinition')
    if friendly_div:
        # Lấy các định nghĩa cơ bản
        basic_meanings = friendly_div.find_all('div', class_='meaning-value')
        for meaning in basic_meanings:
            text = clean_text(meaning)
            if text:
                descriptions.append(text)
        
        # Lấy các ví dụ
        examples = friendly_div.find_all('li', class_='example')
        for example in examples:
            text = clean_text(example)
            if text:
                descriptions.append(f"Ví dụ: {text}")

    # Tìm phần academicDefinition (nếu có)
    academic_div = soup.find('div', id='academicDefinition')
    if academic_div:
        meanings = academic_div.find_all('div', class_='meaning-value')
        for meaning in meanings:
            text = clean_text(meaning)
            if text:
                descriptions.append(f"Định nghĩa học thuật: {text}")

    # Nếu không tìm thấy mô tả, trả về thông báo
    if descriptions:
        return descriptions

# Ví dụ về từ cần tra
word = "mèo"
descriptions = extract_description_from_vdict(word)

# In các mô tả
if descriptions:
    print(f"Mô tả của '{word}':")
    for desc in descriptions:
        print(desc)
else:
    print(f"Không tìm thấy mô tả cho '{word}'")

Mô tả của 'mèo':
Từ " mèo " trong tiếng Việt có nhiều nghĩa và cách sử dụng khác nhau . Dưới đây là một số giải thích và ví dụ giúp bạn hiểu rõ hơn về từ này .
Mèo là một loài động vật nhỏ , thuộc họ mèo (Felidae), có họ hàng gần với hổ , báo . Mèo thường được nuôi trong nhà để bắt chuột và làm bạn .
Cách sử dụng nâng cao :
Biến thể và từ đồng nghĩa :
Các từ gần giống có thể là " chó ", " hổ ", " báo ", vì chúng đều thuộc họ động vật ăn thịt . Tuy nhiên , " chó " thường được nuôi làm thú cưng và có vai trò khác với " mèo ".
Từ đồng nghĩa có thể là " mèo hoang " khi chỉ những con mèo không được nuôi dưỡng , sống tự nhiên ở bên ngoài .
Từ " mèo " trong tiếng Việt không chỉ đơn thuần là một loài động vật mà còn mang nhiều ý nghĩa và sắc thái khác nhau trong ngữ cảnh giao tiếp .
Ví dụ: Ví dụ : " Nhà tôi có một con mèo rất dễ thương . Nó thường bắt chuột trong bếp ."
Ví dụ: Trong tiếng Việt, có nhiều thành ngữ và cụm từ có chứa từ " mèo ". Ví dụ :
Ví dụ: Từ " mèo " còn có một nghĩa khác tro

In [46]:
def escape_special_chars(text):
    text = text.replace("'", "\\'")
    text = text.replace('"', '\\"')
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    return text

# Hàm tạo Node mô tả duy nhất trong Neo4j
def create_description_node(tx, descriptions, entity):
    full_description = " ".join(descriptions)
    full_description = escape_special_chars(full_description)
    
    max_length = 4000
    if len(full_description) > max_length:
        full_description = full_description[:max_length]
    
    query_create = f"""
    MERGE (d:Description {{text: '{full_description}'}})
    WITH d
    MATCH (e:Entity {{name: '{entity}'}})
    MERGE (e)-[:HAS_DESCRIPTION]->(d)
    """
    tx.run(query_create)

# Lấy danh sách thực thể từ Neo4j
def get_entities_from_neo4j():
    with driver.session() as session:
        result = session.run("MATCH (e:Entity) RETURN e.name AS entity")
        return [record["entity"] for record in result]

# Trích xuất mô tả và tạo Node mô tả duy nhất trong Neo4j
def link_entities_to_descriptions():
    entities = get_entities_from_neo4j()  # Lấy danh sách thực thể từ Neo4j
    for entity in entities:
        descriptions = extract_description_from_vdict(entity)  # Trích xuất mô tả từ Vdict
        
        if descriptions:  # Chỉ tạo node nếu tìm thấy mô tả
            with driver.session() as session:
                with session.begin_transaction() as tx:
                    create_description_node(tx, descriptions, entity)
            print(f"Đã tạo mô tả cho thực thể '{entity}'")
        else:
            print(f"Không tìm thấy mô tả cho thực thể '{entity}', bỏ qua.")
link_entities_to_descriptions()

Không tìm thấy mô tả cho thực thể 'khu phố', bỏ qua.
Đã tạo mô tả cho thực thể 'khuya'
Đã tạo mô tả cho thực thể 'nhà thờ'
Không tìm thấy mô tả cho thực thể 'trung học cơ sở', bỏ qua.
Không tìm thấy mô tả cho thực thể 'bãi tắm', bỏ qua.
Không tìm thấy mô tả cho thực thể 'đại sứ quán', bỏ qua.
Đã tạo mô tả cho thực thể 'quân sự'
Đã tạo mô tả cho thực thể 'quản lý'
Đã tạo mô tả cho thực thể 'tập đoàn'
Không tìm thấy mô tả cho thực thể 'đường mòn', bỏ qua.
Không tìm thấy mô tả cho thực thể 'ôn gia bảo', bỏ qua.
Đã tạo mô tả cho thực thể 'nhựa'
Đã tạo mô tả cho thực thể 'lâm'
Không tìm thấy mô tả cho thực thể 'vàng bạc', bỏ qua.
Không tìm thấy mô tả cho thực thể 'da giày', bỏ qua.
Đã tạo mô tả cho thực thể 'sở tại'
Đã tạo mô tả cho thực thể 'rượu'
Không tìm thấy mô tả cho thực thể 'xuân diệu', bỏ qua.
Không tìm thấy mô tả cho thực thể 'fisheries', bỏ qua.
Không tìm thấy mô tả cho thực thể 'làm vườn', bỏ qua.
Đã tạo mô tả cho thực thể 'tiểu khu'
Không tìm thấy mô tả cho thực thể 'dinh thống

In [47]:
# Lấy danh sách file trong thư mục vi-wordnet
wordnet_files = glob.glob("vi-wordnet/*.csv")
print(f"📂 Tìm thấy {len(wordnet_files)} file VNWordNet.")

# Đọc tất cả các nhóm từ đồng nghĩa từ VNWordNet
vnwordnet_synonym_groups = []
vnwordnet_entities = set()  # Để lưu tất cả từ trong từ điển, tránh trùng lặp

for file in wordnet_files:
    lines = load_synonym_file(file)
    for line in lines:
        # Chuẩn hóa và bỏ trống
        group = {normalize_token(word, preserve_capitalization=False) for word in line.split(",") if word.strip()}
        if len(group) > 1:  # Đảm bảo ít nhất có 2 từ đồng nghĩa trở lên
            vnwordnet_synonym_groups.append(list(group))
            vnwordnet_entities.update(group)

print(f"✅ Tổng số nhóm từ đồng nghĩa: {len(vnwordnet_synonym_groups)}")
print(f"✅ Tổng số từ duy nhất từ VNWordNet: {len(vnwordnet_entities)}")
print(f"🔍 Ví dụ 5 nhóm từ đồng nghĩa:", vnwordnet_synonym_groups[:5])

📂 Tìm thấy 47 file VNWordNet.
✅ Tổng số nhóm từ đồng nghĩa: 28966
✅ Tổng số từ duy nhất từ VNWordNet: 59435
🔍 Ví dụ 5 nhóm từ đồng nghĩa: [['máy quay phim', 'máy thu hình', 'máy ghi hình'], ['ca', 'li có quai', 'cốc có quai'], ['kính thiên văn', 'kính viễn vọng'], ['bản vẽ kĩ thuật', 'bản vẽ cấu trúc', 'bản thiết kế', 'thiết kế'], ['đàn clarinet', 'đàn clarinét']]


In [48]:
def create_synonym_relationship(tx, original_entity, synonym):
    """
    Tạo cạnh SYNONYM giữa thực thể gốc và từ đồng nghĩa.
    - Chỉ tạo cạnh nếu cả hai thực thể đã tồn tại trong đồ thị Neo4j.
    """
    query = """
    MATCH (a:Entity {name: $original_entity}), (b:Entity {name: $synonym})
    WHERE a IS NOT NULL AND b IS NOT NULL
    MERGE (a)-[:SYNONYM]->(b)
    MERGE (b)-[:SYNONYM]->(a)
    """
    tx.run(query, original_entity=original_entity, synonym=synonym)
with driver.session() as session:
    for entity in corpus_entities:  # Thực thể đã chuẩn hóa trước đó
        # Tìm từ đồng nghĩa trực tiếp từ VNWordNet
        synonyms = [syn for group in vnwordnet_synonym_groups 
                    if entity in group  # Không chuẩn hóa lại nữa
                    for syn in group if syn != entity]

        # Chèn cạnh SYNONYM
        for syn in synonyms:
            session.execute_write(create_synonym_relationship, entity, syn)

print("✅ Đã tạo cạnh SYNONYM mà không thay đổi thực thể gốc.")


✅ Đã tạo cạnh SYNONYM mà không thay đổi thực thể gốc.


In [52]:
# Hàm tạo mối quan hệ COOCCURRENCE
def create_cooccurrence_relationship(tx, word1, word2, weight):
    query = """
    MATCH (a:Entity {name: $word1}), (b:Entity {name: $word2})
    WHERE a IS NOT NULL AND b IS NOT NULL
    MERGE (a)-[r:COOCCURRENCE]->(b)
    SET r.weight = $weight
    """
    tx.run(query, word1=word1, word2=word2, weight=weight)

# Hàm tạo mối quan hệ CONTEXT
def create_context_relationship(tx, word1, word2, similarity):
    query = """
    MATCH (a:Entity {name: $word1}), (b:Entity {name: $word2})
    MERGE (a)-[r:CONTEXT]->(b)
    SET r.similarity = $similarity
    """
    tx.run(query, word1=word1, word2=word2, similarity=similarity)

# Hàm lấy thực thể và embedding từ Neo4j
def get_entities_with_embeddings(tx, sample_size):
    query = """
    MATCH (n:Entity)
    RETURN n.name AS name, n.embedding AS embedding
    LIMIT $sample_size
    """
    results = tx.run(query, sample_size=sample_size)
    
    entities = []
    embeddings = []
    for record in results:
        name = record["name"]
        embedding = np.array(record["embedding"], dtype=np.float32) if record["embedding"] else None
        if embedding is not None:
            entities.append(name)
            embeddings.append(embedding)
    
    return entities, embeddings

# Bước 1: Tính đồng hiện
co_occurrence = defaultdict(int)
for line in corpus_lines:
    ents = list(set(extract_entities_vn(line))) 
    for i in range(len(ents)):
        for j in range(i+1, len(ents)):
            pair = tuple(sorted([ents[i], ents[j]]))
            co_occurrence[pair] += 1

filtered_cooccurrence = {k: v for k, v in co_occurrence.items() if v > 2}

# Bước 2: Lấy thực thể và embedding từ Neo4j
with driver.session() as session:
    entities, embeddings = session.execute_read(get_entities_with_embeddings, sample_size=5000)

# Bước 3: Tính độ tương đồng embedding và kết hợp với đồng hiện
co_context_pairs = []
for (word1, word2), freq in filtered_cooccurrence.items():
    if word1 in entities and word2 in entities:
        idx1 = entities.index(word1)
        idx2 = entities.index(word2)
        similarity = 1 - cosine(embeddings[idx1], embeddings[idx2])
        if similarity > 0.75:  # Kết hợp điều kiện: đồng hiện cao và độ tương đồng cao
            co_context_pairs.append((word1, word2, freq, similarity))

# Bước 4: Tạo mối quan hệ COOCCURRENCE và CONTEXT trong Neo4j
with driver.session() as session:
    # Tạo COOCCURRENCE
    for (word1, word2), freq in filtered_cooccurrence.items():
        session.execute_write(create_cooccurrence_relationship, word1, word2, freq)
    
    # Tạo CONTEXT cho các cặp thỏa mãn cả hai điều kiện
    for word1, word2, freq, similarity in co_context_pairs:
        session.execute_write(create_context_relationship, word1, word2, similarity)

print(f"✅ Đã tạo {len(co_context_pairs)} cặp đồng ngữ cảnh (dựa trên đồng hiện và embedding).")

# Đóng kết nối
driver.close()

✅ Đã tạo 5 cặp đồng ngữ cảnh (dựa trên đồng hiện và embedding).


In [53]:
def extract_syntax_relations(text):
    """
    Dùng VnCoreNLP để trích xuất các quan hệ cú pháp giữa các từ.
    Chỉ lấy các từ có POS thuộc nhóm quan trọng.
    """
    annotations = model_vn.annotate_text(text)
    syntax_relations = []
    
    for sent in annotations.values():
        for token in sent:
            head_idx = token["head"] - 1  # Chỉ số từ đứng đầu (VnCoreNLP bắt đầu từ 1)
            pos_tag = token["posTag"]  # Lấy POS của từ

            # Kiểm tra xem có thuộc nhóm POS cần thiết không
            if head_idx >= 0 and pos_tag in VALID_POS_TAGS:
                head_word = sent[head_idx]["wordForm"]  # Lấy từ đứng đầu

                # Kiểm tra head_word có bị rỗng hay không trước khi thêm vào danh sách
                if head_word.strip():
                    syntax_relations.append((token["wordForm"], head_word, token["depLabel"], pos_tag))

    return syntax_relations


def add_syntax_relationship(tx, dependent, head, relation):
    """
    Tạo cạnh SYNTAX giữa dependent và head trong Neo4j.
    Chỉ lưu nếu cả dependent và head đều đã tồn tại trong đồ thị.
    """
    query = """
    MATCH (a:Entity {name: $head}), (b:Entity {name: $dependent})
    MERGE (a)-[r:SYNTAX {relation: $relation}]->(b)
    """
    tx.run(query, dependent=dependent, head=head, relation=relation)

# 💾 Duyệt qua từng câu trong corpus và trích xuất quan hệ cú pháp
syntax_edges = []
for line in corpus_lines:
    relations = extract_syntax_relations(line)
    syntax_edges.extend(relations)

print(f"✅ Đã trích xuất {len(syntax_edges)} quan hệ cú pháp.")
# 🔹 Chỉ lấy 2000 cạnh đầu tiên để lưu vào Neo4j
syntax_edges_sample = syntax_edges[:3000]

# Lưu vào Neo4j (Chỉ tạo cạnh nếu cả hai thực thể đã tồn tại)
with driver.session() as session:
    count = 0
    for dependent, head, relation, pos_tag in syntax_edges_sample:
        session.execute_write(add_syntax_relationship, dependent, head, relation)
        count += 1

print(f"✅ Đã lưu {count} quan hệ cú pháp.")


✅ Đã trích xuất 1159938 quan hệ cú pháp.


/var/folders/13/t8vndqp931b5g_jv43m0b3600000gn/T/ipykernel_84630/3511536930.py:47: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


✅ Đã lưu 3000 quan hệ cú pháp.


In [57]:
def get_entities_from_neo4j(tx):
    """ Lấy danh sách thực thể từ Neo4j """
    query = "MATCH (n:Entity) RETURN n.name AS name"
    return [record["name"] for record in tx.run(query)]

def create_sense_relationships(tx, word):
    """ Tạo quan hệ SENSE_OF giữa từ đơn và các thực thể dài hơn """
    query = """
    MATCH (w:Entity {name: $word})
    MATCH (e:Entity) WHERE e.name CONTAINS $word AND e.name <> $word
    MERGE (w)-[:SENSE_OF]->(e)
    """
    tx.run(query, word=word)

# Quy trình chính
with driver.session() as session:
    entities = session.execute_read(get_entities_from_neo4j)

with driver.session() as session:
    for word in entities:
        session.execute_write(create_sense_relationships, word)

print("✅ Đã tạo mối quan hệ SENSE_OF trong Neo4j!")

/var/folders/13/t8vndqp931b5g_jv43m0b3600000gn/T/ipykernel_84630/647249672.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/13/t8vndqp931b5g_jv43m0b3600000gn/T/ipykernel_84630/647249672.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


✅ Đã tạo mối quan hệ SENSE_OF trong Neo4j!
